<a href="https://colab.research.google.com/github/Utpal-Mishra/Omdena-Berlin-Chapter-2023/blob/main/OmdenaBerlinChapter2023Part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/Omdena/Berlin/Train Delay Prediction/Train Delays/2023/Data_raw_punctuality_202301.csv'
data = pd.read_csv(path)
print('Date Shape: {}'.format(data.shape))

data.tail()

Date Shape: (1978735, 19)


,DATDEP,TRAIN_NO,RELATION,TRAIN_SERV,PTCAR_NO,LINE_NO_DEP,REAL_TIME_ARR,REAL_TIME_DEP,PLANNED_TIME_ARR,PLANNED_TIME_DEP,DELAY_ARR,DELAY_DEP,RELATION_DIRECTION,PTCAR_LG_NM_NL,LINE_NO_ARR,PLANNED_DATE_ARR,PLANNED_DATE_DEP,REAL_DATE_ARR,REAL_DATE_DEP
1978730,31JAN2023,17201,EXTRA,SNCB/NMBS,157,34,19:46:53,19:47:42,19:47:00,19:48:00,-6.0,-17.0,NaN,BILZEN,34,31JAN2023,31JAN2023,31JAN2023,31JAN2023
1978731,31JAN2023,17201,EXTRA,SNCB/NMBS,233,34,19:48:45,19:48:45,19:49:00,19:49:00,-15.0,-15.0,NaN,BILZEN-WIJKSPOOR,34,31JAN2023,31JAN2023,31JAN2023,31JAN2023
1978732,31JAN2023,17201,EXTRA,SNCB/NMBS,324,34,19:53:32,19:55:02,19:54:00,19:55:00,-27.0,2.0,NaN,DIEPENBEEK,34,31JAN2023,31JAN2023,31JAN2023,31JAN2023
1978733,31JAN2023,17201,EXTRA,SNCB/NMBS,523,NaN,20:00:49,NaN,20:03:00,NaN,-130.0,NaN,NaN,HASSELT,34,31JAN2023,NaN,31JAN2023,NaN
1978734,31JAN2023,18742,EXTRA,SNCB/NMBS,70,NaN,19:37:30,NaN,19:31:00,NaN,390.0,NaN,NaN,ARLON,162,31JAN2023,NaN,31JAN2023,NaN


In [ ]:
# data.info()
data.isnull().sum()

DATDEP                     0
TRAIN_NO                   0
RELATION                   0
TRAIN_SERV                 0
PTCAR_NO                   0
LINE_NO_DEP            94606
REAL_TIME_ARR          94652
REAL_TIME_DEP          94533
PLANNED_TIME_ARR       94652
PLANNED_TIME_DEP       94533
DELAY_ARR              94648
DELAY_DEP              94529
RELATION_DIRECTION    103312
PTCAR_LG_NM_NL             0
LINE_NO_ARR            94724
PLANNED_DATE_ARR       94652
PLANNED_DATE_DEP       94533
REAL_DATE_ARR          94652
REAL_DATE_DEP          94533
dtype: int64

In [ ]:
# data.drop(columns = ['LINE_NO_DEP', 'LINE_NO_ARR'], axis=1, inplace=True)
data.dropna(subset = ['RELATION_DIRECTION', 'REAL_TIME_ARR', 'LINE_NO_DEP', 'LINE_NO_ARR'], inplace = True)

print('Total Null Values: {}'.format(data.isnull().sum().sum()))

Total Null Values: 0


In [ ]:
#
data.REAL_TIME_ARR    = pd.to_datetime(data.REAL_TIME_ARR, format = '%H:%M:%S').dt.time
data.REAL_TIME_DEP    = pd.to_datetime(data.REAL_TIME_DEP, format = '%H:%M:%S').dt.time
data.PLANNED_TIME_ARR = pd.to_datetime(data.PLANNED_TIME_ARR, format = '%H:%M:%S').dt.time
data.PLANNED_TIME_DEP = pd.to_datetime(data.PLANNED_TIME_DEP, format = '%H:%M:%S').dt.time

#
data.DATDEP           = pd.to_datetime(data.DATDEP, format = '%d%b%Y')
data.PLANNED_DATE_ARR = pd.to_datetime(data.PLANNED_DATE_ARR, format = '%d%b%Y')
data.PLANNED_DATE_DEP = pd.to_datetime(data.PLANNED_DATE_DEP, format = '%d%b%Y')
data.REAL_DATE_ARR    = pd.to_datetime(data.REAL_DATE_ARR, format = '%d%b%Y')
data.REAL_DATE_DEP    = pd.to_datetime(data.REAL_DATE_DEP, format = '%d%b%Y')

#
data.TRAIN_NO   = pd.Categorical(data.TRAIN_NO)
data.RELATION   = pd.Categorical(data.RELATION)
data.TRAIN_SERV = pd.Categorical(data.TRAIN_SERV)

In [ ]:
data.tail()

,DATDEP,TRAIN_NO,RELATION,TRAIN_SERV,PTCAR_NO,LINE_NO_DEP,REAL_TIME_ARR,REAL_TIME_DEP,PLANNED_TIME_ARR,PLANNED_TIME_DEP,DELAY_ARR,DELAY_DEP,RELATION_DIRECTION,PTCAR_LG_NM_NL,LINE_NO_ARR,PLANNED_DATE_ARR,PLANNED_DATE_DEP,REAL_DATE_ARR,REAL_DATE_DEP
1978649,2023-01-31,5466,L L2,SNCB/NMBS,726,37A,17:02:54,17:06:13,17:02:00,17:06:00,54.0,13.0,L L2: LIERS -> FLEMALLE-HAUTE,LIEGE-GUILLEMINS,34,2023-01-31,2023-01-31,2023-01-31,2023-01-31
1978650,2023-01-31,5466,L L2,SNCB/NMBS,358,125A,17:11:15,17:11:15,17:11:00,17:11:00,15.0,15.0,L L2: LIERS -> FLEMALLE-HAUTE,KINKEMPOIS-RELAIS,125A,2023-01-31,2023-01-31,2023-01-31,2023-01-31
1978651,2023-01-31,5466,L L2,SNCB/NMBS,2011,125A,17:13:30,17:14:11,17:14:00,17:14:00,-29.0,11.0,L L2: LIERS -> FLEMALLE-HAUTE,OUGREE,125A,2023-01-31,2023-01-31,2023-01-31,2023-01-31
1978652,2023-01-31,5466,L L2,SNCB/NMBS,1067,125A,17:16:17,17:17:16,17:17:00,17:17:00,-42.0,16.0,L L2: LIERS -> FLEMALLE-HAUTE,SERAING,125A,2023-01-31,2023-01-31,2023-01-31,2023-01-31
1978653,2023-01-31,5466,L L2,SNCB/NMBS,428,125A,17:21:09,17:21:09,17:21:00,17:21:00,9.0,9.0,L L2: LIERS -> FLEMALLE-HAUTE,FLEMALLE-ANNEXE,125A,2023-01-31,2023-01-31,2023-01-31,2023-01-31


In [ ]:
data['ORIGIN']      = data.RELATION_DIRECTION.apply(lambda x: x.split(':')[1].split(' ->')[0])
data['DESTINATION'] = data.RELATION_DIRECTION.apply(lambda x: x.split(':')[1].split('> ')[1])
data['WEEKDAY']     = data['REAL_DATE_ARR'].apply(lambda x: x.strftime('%A'))

data.head()

,DATDEP,TRAIN_NO,RELATION,TRAIN_SERV,PTCAR_NO,LINE_NO_DEP,REAL_TIME_ARR,REAL_TIME_DEP,PLANNED_TIME_ARR,PLANNED_TIME_DEP,...,RELATION_DIRECTION,PTCAR_LG_NM_NL,LINE_NO_ARR,PLANNED_DATE_ARR,PLANNED_DATE_DEP,REAL_DATE_ARR,REAL_DATE_DEP,ORIGIN,DESTINATION,WEEKDAY
0,2023-01-01,10,ICE,SNCB/NMBS,825,37,20:28:12,20:28:12,20:29:00,20:29:00,...,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,HERGENRATH,37,2023-01-01,2023-01-01,2023-01-01,2023-01-01,FRANKFURT(MAIN) HBF,BRUSSEL-ZUID,Sunday
1,2023-01-01,10,ICE,SNCB/NMBS,266,37,20:40:26,20:40:26,20:40:00,20:40:00,...,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,CHENEE,3,2023-01-01,2023-01-01,2023-01-01,2023-01-01,FRANKFURT(MAIN) HBF,BRUSSEL-ZUID,Sunday
2,2023-01-01,10,ICE,SNCB/NMBS,27,37,20:41:38,20:41:38,20:41:00,20:41:00,...,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,ANGLEUR,37,2023-01-01,2023-01-01,2023-01-01,2023-01-01,FRANKFURT(MAIN) HBF,BRUSSEL-ZUID,Sunday
3,2023-01-01,10,ICE,SNCB/NMBS,726,36,20:44:02,20:47:05,20:44:00,20:46:00,...,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,LIEGE-GUILLEMINS,37,2023-01-01,2023-01-01,2023-01-01,2023-01-01,FRANKFURT(MAIN) HBF,BRUSSEL-ZUID,Sunday
4,2023-01-01,10,ICE,SNCB/NMBS,31,2,20:52:45,20:52:45,20:51:00,20:51:00,...,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,ANS,36,2023-01-01,2023-01-01,2023-01-01,2023-01-01,FRANKFURT(MAIN) HBF,BRUSSEL-ZUID,Sunday


In [ ]:
print('Total No.of Trains: {}'.format(len(data.TRAIN_NO.unique())))

Total No.of Trains: 3488


In [ ]:
data.TRAIN_NO.value_counts()

3806    1890
3831    1887
3814    1866
3816    1861
3820    1855
        ... 
9991       1
9992       1
9410       1
9110       1
9892       1
Name: TRAIN_NO, Length: 3488, dtype: int64

### **DATE**

In [ ]:
print('TRAIN DELAYS in ARRIVAL   : {} Seconds :: {} Hours'.format(round(data.groupby('DATDEP')['DELAY_ARR'].mean().sum(), 3), str(timedelta(seconds=data.groupby('DATDEP')['DELAY_ARR'].mean().sum()))))
print('TRAIN DELAYS in DEPARTURE : {} Seconds :: {} Hours'.format(round(data.groupby('DATDEP')['DELAY_DEP'].mean().sum(), 3), str(timedelta(seconds=data.groupby('DATDEP')['DELAY_DEP'].mean().sum()))))

TRAIN DELAYS in ARRIVAL   : 3776.365 Seconds :: 1:02:56.364525 Hours
TRAIN DELAYS in DEPARTURE : 4359.757 Seconds :: 1:12:39.756559 Hours


In [ ]:
delayARR = pd.DataFrame(data.groupby('DATDEP')['DELAY_ARR'].mean()).reset_index()

fig = px.bar(delayARR, x = delayARR.DATDEP, y = delayARR.DELAY_ARR, color = delayARR.DELAY_ARR)
fig.update_xaxes(rangeslider_visible=False, showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_layout(height=500, width=1200, xaxis_title="Date", yaxis_title="Train Delay (in Seconds)", title_text="DELAYS IN TRAIN ARRIAL (Samples)") 
fig.show()

In [ ]:
delayDEP = pd.DataFrame(data.groupby('DATDEP')['DELAY_DEP'].mean()).reset_index()

fig = px.bar(delayDEP, x = delayDEP.DATDEP, y = delayDEP.DELAY_DEP, color = delayDEP.DELAY_DEP)
fig.update_xaxes(rangeslider_visible=False, showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_layout(height=500, width=1200, xaxis_title="Date", yaxis_title="Train Delay (in Seconds)", title_text="DELAYS IN TRAIN DEPARTURE") 
fig.show()

In [ ]:
print('Average Time Delay: {}'.format(data.groupby('DATDEP')['DELAY_ARR'].mean()))

Average Time Delay: DATDEP
2023-01-01     69.526366
2023-01-02     72.057409
2023-01-03     84.309532
2023-01-04    122.712372
2023-01-05     88.659110
2023-01-06     83.593472
2023-01-07     78.689716
2023-01-08     72.160302
2023-01-09    170.529795
2023-01-10    162.157898
2023-01-11    128.731325
2023-01-12    196.773066
2023-01-13    163.794956
2023-01-14    115.609847
2023-01-15     62.194599
2023-01-16    164.607574
2023-01-17    132.822262
2023-01-18    171.612039
2023-01-19    151.731964
2023-01-20    153.490746
2023-01-21     81.286492
2023-01-22     91.769354
2023-01-23    160.438155
2023-01-24    137.462461
2023-01-25    140.834480
2023-01-26    143.466743
2023-01-27    114.943010
2023-01-28     90.831957
2023-01-29     86.661975
2023-01-30    140.961523
2023-01-31    141.944025
Name: DELAY_ARR, dtype: float64
